In [1]:
import os
import random
import pandas as pd
import numpy as np
import torch
from torchvision import transforms
from typing import List, Set
from datetime import datetime
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from deep_miml.models import Average, Attention
from deep_miml.train import train_miml_model, test_multi_instance_model
from deep_miml.utils import precision_recall_helper, get_avg_batch_precision_recall_at_k, timing
import torchvision.transforms as transforms
import torch.optim as optim
from pathlib import Path

torch.manual_seed(123)
torch.cuda.manual_seed(123)
np.random.seed(123)
random.seed(123)
torch.backends.cudnn.enabled=False
torch.backends.cudnn.deterministic=True

/home/mridula/Projects/MIML/notebooks


In [2]:
class YelpDataset:
    def __init__(self, business_df, image_df, image_folder, transform):
        self.business_df = business_df
        self.image_df = image_df
        self.transform = transform
        self.image_folder = image_folder
        
    
    def __getitem__(self, i: int):
        b_id = self.business_df.iloc[i]['business_id']
        imgs = list(self.image_df[self.image_df['business_id'] == b_id]['photo_id'])
        
        imgs = [self.transform(Image.open(self.image_folder+f"{img}.jpg")) for img in imgs]
        
        labels = torch.FloatTensor(self.business_df.iloc[i,1:])
        
        return imgs, labels
        
    def __len__(self):
        return self.business_df.shape[0]
     
        
def collate_fn(batch, input_size=224):
    img_lists, bag_labels = zip(*batch)

    imgs = [img for img_list in img_lists for img in img_list]

    sizes = torch.LongTensor([len(img_list) for img_list in img_lists])

    bag_labels = torch.stack(bag_labels)
    

    if len(imgs) != 0:
        imgs = torch.stack(imgs)
    else:
        imgs = torch.zeros((0, 3, input_size, input_size)).float()

    return imgs, sizes, bag_labels
        

In [3]:
images_folder_path = '../../WIML/yelp_data/website_data/yelp_photos/photos/'
final_business_df = pd.read_csv('../../WIML/yelp_data/website_data/final_business_data.csv')
final_images_df = pd.read_csv('../../WIML/yelp_data/website_data/final_images_data.csv')

print(f'Shape of business_df {final_business_df.shape}')
print(f'Shape of images_df {final_images_df.shape}')

Shape of business_df (23137, 9)
Shape of images_df (139443, 2)


In [12]:
full_business_list = final_images_df.groupby(['business_id'])['photo_id'].count().reset_index()
full_business_list = full_business_list[full_business_list.photo_id > 1]
print(full_business_list.shape)
full_business_list = full_business_list[full_business_list.photo_id < 50]
print(full_business_list.shape)

(16785, 2)
(16561, 2)


In [13]:
# Save the results for reproducability 
full_business_list = set(full_business_list.business_id)

test_business_list = random.sample(full_business_list,500)
val_business_list = random.sample(full_business_list.difference(test_business_list), 1000)
train_business_list = full_business_list.difference(test_business_list+val_business_list)

In [14]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


image_datasets = {}

image_datasets['train'] = YelpDataset(business_df= final_business_df[final_business_df.business_id.isin(train_business_list)],
                           image_df= final_images_df,
                           image_folder = images_folder_path,
                           transform= data_transforms['train'])

image_datasets['val'] = YelpDataset(business_df= final_business_df[final_business_df.business_id.isin(val_business_list)],
                         image_df= final_images_df,
                          image_folder = images_folder_path,
                         transform = data_transforms['val'])

image_datasets['test'] = YelpDataset(business_df= final_business_df[final_business_df.business_id.isin(test_business_list)],
                         image_df= final_images_df,
                          image_folder = images_folder_path,
                         transform = data_transforms['test'])

# print( f'len of train dataset {train_dataset.__len__()}')
# print( f'len of val dataset {val_dataset.__len__()}')
# train_dataset.__getitem__(0)

# DataLoaders

In [15]:
model_name = 'resnet18'
num_categories = 8
model_type = 'avg'
lr = 0.0001

use_pretrained = True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [16]:
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x],
                                                    batch_size= 4,
                                                    shuffle=(x == 'train'),
                                                    num_workers= 12,
                                                    collate_fn=collate_fn)
                    for x in ['train', 'val','test']}

print('DataLoaders ready ... \n')


DataLoaders ready ... 



# Training

In [18]:
def train_and_save(model_type,model_name, lr, device, num_epochs= 1,
                   early_stopping= True, save_folder = '../temp/', patience =5):

    if model_type == 'avg':
        model_ft = Average(num_classes = num_categories,model_name= model_name)
    elif model_type == 'attention':
        model_ft = Attention(num_classes = num_categories, model_name= model_name)

    else:
        print("Enter valid model type")
        exit(0)


        # Send the model to GPU
    model_ft = model_ft.to(device)

        # Observe that all parameters are being optimized
    optimizer_ft = optim.Adam(model_ft.parameters(), lr= lr)

    # You should pass logits to nn.BCEwithLogitsLoss and probabilities (using "sigmoid") to nn.BCELoss.
    # Using BCEWithLogitsLoss because it is more stable 
    criterion = torch.nn.BCEWithLogitsLoss(weight=None, reduction='mean')

        # Train and evaluate
    model_ft = train_miml_model(model=model_ft,
                                device = device,
                                dataloaders=dataloaders_dict,
                                criterion=criterion,
                                optimizer=optimizer_ft,
                                save_folder= save_folder ,
                                num_epochs= num_epochs,
                                early_stopping = early_stopping,
                                patience = patience
                                )

    torch.save(model_ft.state_dict(),
        Path.cwd().joinpath(save_folder,"yelp_{}_{}_pretrained_{}.pt".format(model_name, model_type, use_pretrained)))
    
    return model_ft


In [10]:
torch.cuda.empty_cache()

# Testing

In [ ]:
# TESTING
result = test_multi_instance_model(model_ft, device, dataloaders_dict['test'])